# Setup

In [ ]:
import random
import string
import os

from pylibdmtx.pylibdmtx import encode, decode
from PIL import Image

from ultralytics import YOLO, settings
root_dir = os.getcwd().replace('\\notebooks', '')
settings.update({'datasets_dir': f'{root_dir}/data/module_pose/simple_synth', 'runs_dir': f'{root_dir}/yolo/runs'})
print(root_dir)

In [ ]:
# Should print True if GPU is available for use
import torch
print(torch.cuda.is_available())

# Simple Synthesis

Synthesizing a simple DMC dataset for the ultralytics yolo11 pose model to use.

In [ ]:
def gen_string():
    '''
    Generates a serial number to encode
    
    Serial numbers are:
    - 11 characters long
    - Index 0, 2, 4, 5, 6, 7, 8, 9, 10 are random digits
    - Index 1 and 3 are uppercase letters
    - Index 11, 12, 13, 14 are an incremental number starting from 0001

    Example serial number: 4 L 4 N 0418028 0001
    '''

    to_encode = ''

    # first 11 indexes
    for j in range(11):
        # 1 and 3 are uppercase
        if j in [1, 3]:
            to_encode += random.choice(string.ascii_uppercase)
        else:
            to_encode += str(random.randrange(0, 10))

    # last 4 indexes
    end = str(random.randrange(1, 99))
    if len(end) == 1:
        end = '0' + end
    elif len(end) == 2:
        end = '00' + end
    else:
        end = '000' + end
    to_encode += end

    return to_encode

def encode_image(to_encode):
    '''Creates a PIL image containing DMC encoding of given string'''

    encoded = encode(to_encode.encode('utf8'))
    img = Image.frombytes('RGB', (encoded.width, encoded.height), encoded.pixels).convert('L')

    # crop image to remove white borders (have to leave some white border to do decode check)
    # img = img.crop((15, 15, img.width-15, img.height-15))

    # upscale image
    img = img.resize((640, 640), Image.NEAREST)

    return img

def get_pose_coords(img, debug=False):
    '''Returns the coordinates of each module within DMC info zone'''
    padding = 96      # padding around DMC info zone in pixels (we only want inner modules!)
    module_width = 32 # pixel width of each module

    raw_coords = [] # raw pixel centers of each module
    label_info = []

    # class info is always 0
    label_info.append(0)

    # center x y coords of entire image
    label_info.extend([0.5, 0.5])

    # width and height of entire DMC
    label_info.extend([0.8, 0.8])

    for x in range(padding+(module_width)//2, img.width-padding, module_width):
        for y in range(padding+(module_width)//2, img.height-padding, module_width):
            # Add raw pixel coords
            raw_coords.append((x, y))

            # Add class info to label_info

            # normalize pixel coords to 0-1
            x_norm = x / img.width
            y_norm = y / img.height
            if img.getpixel((x, y)) < 128:
                label_info.extend([x_norm, y_norm, 2]) # 2 for labeled and visible

                # paint pixel-to-check white for viz / debug
                if debug:
                    img.putpixel((x, y), (255))

            else:
                label_info.extend([x_norm, y_norm, 2]) # 1 for labeled but not visible (maybe try 0 for unlabeled (not in view))

                # paint pixel-to-check black for viz / debug
                if debug:
                    img.putpixel((x, y), (0))

    # convert to single string
    label_info = ' '.join([str(x) for x in label_info])

    # print(black_modules)
    # print(black_module_coords)
    # print(class_labels)
    return raw_coords, label_info

In [ ]:
# testing
test = gen_string()
img = encode_image(test)
raw_coords, label_info = get_pose_coords(img, debug=True)
print(raw_coords)
print(label_info)
img

In [ ]:
def gen_save(type):
    '''Generates a random serial number, encodes it into a DMC image, and saves it to train/val/test folders'''

    to_encode = gen_string()
    img = encode_image(to_encode)

    # save image
    img.save(f'../data/module_pose/simple_synth/images/{type}/{to_encode}.png')

    # get pose coordinates
    raw_coords, label_info = get_pose_coords(img)

    # save pose coordinates
    with open(f'../data/module_pose/simple_synth/labels/{type}/{to_encode}.txt', 'w') as f:
        f.write(f'{label_info}\n')

    return

def delete_old():
    '''Deletes all images and labels in train/val/test folders'''

    for folder in ['train', 'val', 'test']:
        for file in os.listdir(f'../data/module_pose/simple_synth/images/{folder}'):
            os.remove(f'../data/module_pose/simple_synth/images/{folder}/{file}')

        for file in os.listdir(f'../data/module_pose/simple_synth/labels/{folder}'):
            os.remove(f'../data/module_pose/simple_synth/labels/{folder}/{file}')
    
    # delete cache too (if it exists)
    if os.path.exists('../data/module_pose/simple_synth/labels/train.cache'):
        os.remove('../data/module_pose/simple_synth/labels/train.cache')
        os.remove('../data/module_pose/simple_synth/labels/val.cache')

    return

In [ ]:
# delete old images and labels
delete_old()

In [ ]:
# generating train/val/test datasets
n_train = 800
n_val = 100
n_test = 100

# train
for i in range(n_train - len(os.listdir('../data/module_pose/simple_synth/images/train'))):
    gen_save('train')

# val
for i in range(n_val - len(os.listdir('../data/module_pose/simple_synth/images/val'))):
    gen_save('val')

# test
for i in range(n_test - len(os.listdir('../data/module_pose/simple_synth/images/test'))):
    gen_save('test')

# Loading untrained model

In [ ]:
model = YOLO('yolo11n-pose.yaml', task='pose')

# Training
Notes on trained models
- train: first model trained where white modules were labeled as 1 (labeled but not visible) and black modules 2 (labeled and visible)
- train2: first model trained where all modules were labeled as 2 (fixed no gpu)
- train3: same as above but with added pose=20 to see if it is the same as train2
- train4: same as above but with pose=40 to increase focus on pose estimation
- train5: same as above but with pose=100
- train6: same as above but with pose=40? train4 performed better than both train3 and train5
- train7: same as above but with pose=50
- train8: pose=30
- train9: pose=40 no patience (full 100 epochs)
- train10: pose=100 no patience
- train11: pose=40 no patience 1000 epochs

In [ ]:
results = model.train(
    data=f'{root_dir}\\data\\module_pose\\simple_synth\\data.yml', # path to yaml file which specifies dataset parameters
    epochs=1000,
    imgsz=640,                                         # image size (default 640 for yolo)
    single_cls=True,                                   # single-class training
    patience=0,                                       # early stopping patience (after this many epochs with no improvement stop training)
    pretrained=False,                                  # don't use pre-trained weights
    plots=True,                                        # create plots

    # solving GPU memory issue?
    workers=0,                                         # number of worker threads for data loading (0 reduces memory problems at cost of slower training)
    batch=8,                                           # batch size (default 16, reducing to 8 can help)

    pose=40,                                           # Pose weight (default 20, increase to improve pose accuracy)
)

# Evaluating

In [ ]:
# Loading trained model
model = YOLO('../yolo/runs/pose/train11/weights/last.pt', task='pose')

In [ ]:
# # validation
# metrics = model.val()
# print(metrics)

In [ ]:
# test
test_images = os.listdir('../data/module_pose/simple_synth/images/test')
test_images = [f'../data/module_pose/simple_synth/images/test/{x}' for x in test_images][0]
results = model(test_images)

In [ ]:
for result in results:
    boxes = result.boxes
    keypoints = result.keypoints
    result.show()
    break